# Create a View of the Source Table


In [14]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create View `[Uploadfile].[vw_CurrentSemesterCourseOfferings]` from source table `[Uploadfile].[CurrentSemesterCourseOfferings]` >
-- =============================================

USE QueensClassSchedule
GO

CREATE VIEW [Uploadfile].[vw_CurrentSemesterCourseOfferings] AS
SELECT
    [Semester],
    [Sec],
    [Code],
    [Course (hr, crd)] AS OriginalCourseColumn,
    -- Extract CourseName (text before the first '(')
    LTRIM(RTRIM(LEFT([Course (hr, crd)], CHARINDEX('(', [Course (hr, crd)]) - 1))) AS CourseName,
    -- Extract hr (number between '(' and ',')
    LTRIM(RTRIM(SUBSTRING([Course (hr, crd)],
        CHARINDEX('(', [Course (hr, crd)]) + 1,
        CHARINDEX(',', [Course (hr, crd)]) - CHARINDEX('(', [Course (hr, crd)]) - 1))) AS hr,
    -- Extract crd (number between ',' and ')')
    LTRIM(RTRIM(SUBSTRING([Course (hr, crd)],
        CHARINDEX(',', [Course (hr, crd)]) + 1,
        CHARINDEX(')', [Course (hr, crd)]) - CHARINDEX(',', [Course (hr, crd)]) - 1))) AS crd,
    [Description],
    [Day],
    [Time],
    [Instructor],
    [Location],
    [Enrolled],
    [Limit],
    [Mode of Instruction]
FROM
    [Uploadfile].[CurrentSemesterCourseOfferings];


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.018

# Create `[Udt]` Schema and User-Defined Data Types:

In [15]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski, Jascharan Singh, Sarmad Ali, Augusta Na, Carlos Vega >
-- Create date: < 12/3/2024 >
-- Description:	< Create the `Udt` schema and User-defined Data Types >
-- =============================================

USE QueensClassSchedule
GO

CREATE  SCHEMA  [Udt]
GO

CREATE  TYPE    [Udt].[SurrogateKeyInt]     FROM    [int]               NULL
GO
CREATE  TYPE    [Udt].[DateAdded]           FROM    Datetime2(7)    NOT NULL
GO
CREATE  TYPE    [Udt].[ClassTime]           FROM    nchar(5)        NOT NULL
GO
CREATE  TYPE    [Udt].[ClassHours]          FROM    nchar(5)        NOT NULL
GO
CREATE  TYPE    [Udt].[IndividualProject]   FROM    NVARCHAR(60)    NOT NULL
GO
CREATE  TYPE    [Udt].[LastName]            FROM    NVARCHAR(35)    NOT NULL
GO
CREATE  TYPE    [Udt].[FirstName]           FROM    NVARCHAR(20)    NOT NULL
GO
CREATE  TYPE    [Udt].[GroupName]           FROM    NVARCHAR(20)    NOT NULL
GO
CREATE  TYPE    [Udt].[WorkFlowString]      FROM    NVARCHAR(100)   NOT NULL
GO
CREATE  TYPE    [Udt].[RowCount]            FROM    [int]           NOT NULL
GO
CREATE  TYPE    [Udt].[SurrogateKeyString]  FROM    NVARCHAR(30)    NOT NULL
GO
CREATE  TYPE    [Udt].[ClassLength]         FROM    NVARCHAR(30)    NOT NULL
GO
CREATE  TYPE    [Udt].[SurrogateKeyFloat]   FROM    FLOAT           NOT NULL
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.054

# Create Other Required Schemas

### Create `[UserAuthorization]`, `[DbSecurity]`, `[Course]`, `[Location]`, `[Department]`, `[Process]`, and `[Project3]` schemas:

In [16]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create the required schemas >
-- =============================================

USE QueensClassSchedule
GO

CREATE  SCHEMA  [DbSecurity]
GO
CREATE  SCHEMA  [Course]
GO
CREATE  SCHEMA  [Location]
GO
CREATE  SCHEMA  [Department]
GO
CREATE  SCHEMA  [Process]
GO
CREATE  SCHEMA  [Project3]
GO              

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.022

# Create `[DbSecurity].[UserAuthorization]` Table

### Create and populate the table storing the group members:

In [17]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create the `[DbSecurity].[UserAuthorization]` table >
-- =============================================

USE QueensClassSchedule
GO

CREATE  TABLE   [DbSecurity].[UserAuthorization]    (
    [UserAuthorizationKey]  [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL    PRIMARY KEY CLUSTERED,
    [ClassTime]             [Udt].[ClassTime]                           NULL    DEFAULT ('10:45'),
    [IndividualProject]     [Udt].[IndividualProject]                   NULL    DEFAULT ('Project 3'),
    [GroupMemberLastName]   [Udt].[LastName]                        NOT NULL,
    [GroupMemberFirstName]  [Udt].[FirstName]                       NOT NULL,
    [GroupName]             [Udt].[GroupName]                       NOT NULL    DEFAULT ('Group 2'),
    [DateAdded]             [Udt].[DateAdded]                           NULL    DEFAULT (SYSDATETIME())
);
GO

INSERT  INTO    [DbSecurity].[UserAuthorization]    (GroupMemberLastName,   GroupMemberFirstName)
VALUES
    ('Sarmad',      'Ali'), 
    ('Daniel',      'Gargiullo'), 
    ('Augusta',     'Na'), 
    ('Jascharan',   'Singh'),
    ('Carlos',      'Vega Lemus'), 
    ('Oksana',      'Weigand-Suminski');
GO

Commands completed successfully.

Commands completed successfully.

(6 rows affected)

Total execution time: 00:00:00.017

# Create `[Course].[Class]` Table:

In [18]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create the `[Course].[Class]` table >
-- =============================================

USE QueensClassSchedule
GO

CREATE  TABLE   [Course].[Class]    (
    [ClassID]       [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [Enrollment]    [Udt].[SurrogateKeyInt]                     NULL,
    [Limit]         [Udt].[SurrogateKeyInt]                     NULL,
    [Section]       [Udt].[SurrogateKeyString]                  NULL,
    [ClassCode]     [Udt].[SurrogateKeyString]                  NULL,
    [Days]          [Udt].[SurrogateKeyString]                  NULL,
    [Time]          [Udt].[ClassLength]                         NULL,
    [ClassStart]    [Udt].[ClassHours]                          NULL,
    [ClassEnd]      [Udt].[ClassHours]                          NULL,
    [Hours]         [Udt].[SurrogateKeyFloat]                   NULL,
    [Credits]       [Udt].[SurrogateKeyInt]                     NULL,
    [InstructorID]  [Udt].[SurrogateKeyInt]                     NULL,
    -- Required fields for all tables:
    [UserAuthorizationKey]  [Udt].[SurrogateKeyInt]             NULL,
    [DateAdded]             [Udt].[DateAdded]                   NULL    DEFAULT (SYSDATETIME()),
    [DateOfLastUpdate]      [Udt].[DateAdded]                   NULL    DEFAULT (SYSDATETIME())
);

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.010